##### EEEL 4220 Final Project
## Wind Farm Sizing and Siting on IEEE-14 System

3. 14-bus single-scenario, with wind planning. Next, extend the model to include wind
variables yb and wb,s for a single scenario s∗. Use the following simplified formulation.

In [1]:
# Import libraries
import cvxpy as cp
import numpy as np
import pandas as pd

In [2]:
# Load data
b_df = pd.read_csv('buses.csv')
g_df = pd.read_csv('generators.csv')
l_df = pd.read_csv('branches.csv')
d_df = pd.read_csv('demand_scenarios.csv')
cfs_df = pd.read_csv('wind_cf_scenarios.csv')
y_df = pd.read_csv('wind_candidates.csv')

In [3]:
# Parameters
c0 = g_df['c0'].to_numpy() # quadratic cost coefficients for generator g
c1 = g_df['c1'].to_numpy() # linear cost coefficients for generator g
c2 = g_df['c2'].to_numpy() # fixed cost coefficients for generator g
Pmin = g_df['Pmin_MW'].to_numpy() # minimum generation limits for generator g
Pmax = g_df['Pmax_MW'].to_numpy() # maximum generation limits for generator g
x = l_df['x_pu'].to_numpy() # reactance of transmission line l
fmax = l_df['rateA_MW'].to_numpy() # thermal limit of line l
D = d_df.loc[d_df['scenario_id'] == 1, 'Pd_MW'].to_numpy() # demand at bus b in scenario s
CF = cfs_df['capacity_factor'].iloc[0] # wind capacity factor in scenario s
ybar = pd.merge(b_df, y_df, on='bus_id', how='left')['max_capacity_MW'].fillna(0).to_numpy() # maximum installable wind capacity at bus b (0 if not candidate)
Cinv = 1.5 # investment cost per MW of installed wind capacity ($/MW)

# Decision variables
p = cp.Variable(len(g_df), nonneg=True) # power output of generator g under scenario s
f  = cp.Variable(len(l_df)) # power flow on line l under scenario s
theta = cp.Variable(len(b_df)) # voltage angle at bus b (radians)
w = cp.Variable(len(b_df), nonneg=True) # wind generation at bus b under scenario s
y = cp.Variable(len(b_df), nonneg=True) # installed wind capacity at bus b (MW)

In [4]:
# Initialize incidence matrices
# Generator–bus
# Ggb = 1 if generator g is connected to bus b, 0 otherwise.
G = np.zeros((len(g_df), len(b_df)))

# Line–bus
# Alb = 1 if bus b is the sending (“from”) end of line l, if bus b is the receiving (“to”) end of line l, 0 otherwise.
A = np.zeros((len(l_df), len(b_df)))

# Populate incidence matrices
# Generator–bus
for gen_index in range(len(g_df)):
    bus_id = g_df.loc[gen_index, 'bus_id']
    bus_index = bus_id - 1
    G[gen_index, bus_index] = 1

# Line–bus
for line_index in range(len(l_df)):
    from_bus_id = l_df.loc[line_index, 'from_bus']
    to_bus_id = l_df.loc[line_index, 'to_bus']
    from_bus_index = from_bus_id - 1
    to_bus_index = to_bus_id - 1
    A[line_index, from_bus_index] = 1
    A[line_index, to_bus_index] = -1

In [5]:
# Define constraints
# Initialize an empty constraint set
con = [] 

# power balance
con.append((A.T @ f) + D == (G.T @ p) + w)

# generator output limits
con.append(p <= Pmax)  # maximum generation

# set reference angle
con.append(theta[0] == 0)

# DC power flow
con.append(f == (A @ theta) / x)

# line flow limits
con.append(f <= fmax)  # maximum line flow
con.append(f >= -fmax)  # minimum line flow

# wind generation limits
con.append(w <= y * CF)  # wind generation cannot exceed installed capacity

# wind capacity limits
con.append(y <= ybar)  # maximum installed capacity

In [6]:
# Define objective function - total cost
obj = cp.Minimize(cp.sum(cp.multiply(c2, cp.square(p)) + cp.multiply(c1, p) + c0) 
                  + Cinv * cp.sum(y))

In [7]:
# Solve the problem
prob1 = cp.Problem(obj, con)
prob1.solve(solver = "Gurobi");

Restricted license - for non-production use only - expires 2027-11-29


In [8]:
# results
print("\n The total generation in the system is:")
print((p.value.sum(axis=0) + w.value.sum(axis=0)).round(1))
print("The total demand in the system is:") 
print(D.sum(axis=0).round(1))

print("\n Expected operating cost:")
print(obj.value.round(1))

print("\n Thermal generator dispatch results: ")
GT = pd.DataFrame(p.value.round(1), index = ['G1', 'G2', 'G3', 'G4', 'G5'])
print(GT.round(1))

print("\n Wind dispatch results: ")
WT = pd.DataFrame(w.value.round(1), index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'])
print(WT.round(1))

print("\n Line flow results: ")
LT = pd.DataFrame(f.value.round(1), index = ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20'])
print(LT.round(1))

print("\n Locational Marginal Prices: ")
MCT = pd.DataFrame(con[0].dual_value, index = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9', 'B10', 'B11', 'B12', 'B13', 'B14'])
print(MCT.round(1))


 The total generation in the system is:
233.1
The total demand in the system is:
233.1

 Expected operating cost:
721.2

 Thermal generator dispatch results: 
       0
G1  79.1
G2  42.9
G3  43.5
G4  35.9
G5  31.7

 Wind dispatch results: 
       0
B1   0.0
B2   0.0
B3   0.0
B4   0.0
B5   0.0
B6   0.0
B7   0.0
B8   0.0
B9   0.0
B10  0.0
B11  0.0
B12  0.0
B13  0.0
B14  0.0

 Line flow results: 
        0
F1   56.2
F2   22.9
F3   31.3
F4   23.6
F5   24.6
F6  -10.0
F7  -23.4
F8   -7.8
F9    1.9
F10  17.2
F11  13.9
F12   9.0
F13  20.2
F14 -31.7
F15  23.9
F16  -2.7
F17   1.9
F18 -10.8
F19   3.5
F20  11.5

 Locational Marginal Prices: 
       0
B1   4.4
B2   4.4
B3   4.4
B4   4.4
B5   4.4
B6   4.4
B7   4.4
B8   4.4
B9   4.4
B10  4.4
B11  4.4
B12  4.4
B13  4.4
B14  4.4
